# Homework Starter — Stage 05: Data Storage
Name: 
Date: 

Objectives:
- Env-driven paths to `data/raw/` and `data/processed/`
- Save CSV and Parquet; reload and validate
- Abstract IO with utility functions; document choices

In [22]:
import os, pathlib, datetime as dt
import pandas as pd
from dotenv import load_dotenv

#Directories are created base on the starter
#.env created with DATA\_DIR\_RAW=data/raw and DATA\_DIR\_PROCESSED=data/processed

load_dotenv()
RAW = pathlib.Path(os.getenv('DATA_DIR_RAW', 'data/raw'))
PROC = pathlib.Path(os.getenv('DATA_DIR_PROCESSED', 'data/processed'))
RAW.mkdir(parents=True, exist_ok=True)
PROC.mkdir(parents=True, exist_ok=True)
print('RAW ->', RAW.resolve())
print('PROC ->', PROC.resolve())

RAW -> C:\Users\13466\bootcamp_enhua_zhu\homework\homework5\data\raw
PROC -> C:\Users\13466\bootcamp_enhua_zhu\homework\homework5\data\processed


## 1) Create or Load a Sample DataFrame
You may reuse data from prior stages or create a small synthetic dataset.

In [23]:
#Create and load dataframe

import numpy as np
dates = pd.date_range('2024-01-01', periods=20, freq='D')
df = pd.DataFrame({'date': dates, 'ticker': ['AAPL']*20, 'price': 150 + np.random.randn(20).cumsum()})
df.head()

,date,ticker,price
0,2024-01-01,AAPL,148.639833
1,2024-01-02,AAPL,148.825222
2,2024-01-03,AAPL,149.635434
3,2024-01-04,AAPL,149.878524
4,2024-01-05,AAPL,149.757249


## 2) Save CSV to data/raw/ and Parquet to data/processed/ (TODO)
- Use timestamped filenames.
- Handle missing Parquet engine gracefully.

In [24]:
def ts(): return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

# CSV saved with the method instructed
csv_path = RAW / f"sample_{ts()}.csv"
df.to_csv(csv_path, index=False)
csv_path

# Pyarrow installed and parquet saved as instructed
pq_path = PROC / f"sample_{ts()}.parquet"
try:
    df.to_parquet(pq_path)
except Exception as e:
    print('Parquet engine not available. Install pyarrow or fastparquet to complete this step.')
    pq_path = None
pq_path

WindowsPath('data/processed/sample_20250817-142848.parquet')

## 3) Reload and Validate (TODO)
- Compare shapes and key dtypes.

In [25]:
def validate_loaded(original, reloaded):
    checks = {
        'shape_equal': original.shape == reloaded.shape,
        'date_is_datetime': pd.api.types.is_datetime64_any_dtype(reloaded['date']) if 'date' in reloaded.columns else False,
        'price_is_numeric': pd.api.types.is_numeric_dtype(reloaded['price']) if 'price' in reloaded.columns else False,
    }
    return checks

# Use validate_loaded to check both ways; compare shapes and key dtypes
df_csv = pd.read_csv(csv_path, parse_dates=['date'])
df_pq = pd.read_parquet(pq_path)
try:
    print('CSV validation: ',validate_loaded(df, df_csv))
except Exception as e:
    print('CSV validation fails.', e)
try:
    print('Parquet validation: ',validate_loaded(df, df_pq))
except Exception as e:
    print('Parquet validation fails.', e)

CSV validation:  {'shape_equal': True, 'date_is_datetime': True, 'price_is_numeric': True}
Parquet validation:  {'shape_equal': True, 'date_is_datetime': True, 'price_is_numeric': True}


## 4) Utilities (TODO)
- Implement `detect_format`, `write_df`, `read_df`.
- Use suffix to route; create parent dirs if needed; friendly errors for Parquet.

In [26]:
import typing as t, pathlib

def detect_format(path: t.Union[str, pathlib.Path]):
    s = str(path).lower()
    if s.endswith('.csv'): return 'csv'
    if s.endswith('.parquet') or s.endswith('.pq') or s.endswith('.parq'): return 'parquet'
    raise ValueError('Unsupported format: ' + s)

def write_df(df: pd.DataFrame, path: t.Union[str, pathlib.Path]):
    p = pathlib.Path(path); p.parent.mkdir(parents=True, exist_ok=True)
    fmt = detect_format(p)
    if fmt == 'csv':
        df.to_csv(p, index=False)
    else:
        try:
            df.to_parquet(p)
        except Exception as e:
            raise RuntimeError('Parquet engine not available. Install pyarrow or fastparquet.') from e
    return p

def read_df(path: t.Union[str, pathlib.Path]):
    p = pathlib.Path(path)
    fmt = detect_format(p)
    if fmt == 'csv':
        return pd.read_csv(p, parse_dates=['date']) if 'date' in pd.read_csv(p, nrows=0).columns else pd.read_csv(p)
    else:
        try:
            return pd.read_parquet(p)
        except Exception as e:
            raise RuntimeError('Parquet engine not available. Install pyarrow or fastparquet.') from e

# 
p_csv = RAW / f"util_{ts()}.csv"
p_pq  = PROC / f"util_{ts()}.parquet"
write_df(df, p_csv); read_df(p_csv).head()
try:
    print(write_df(df, p_pq)) #data processed
    print(read_df(p_pq).head()) #data's header shown
except RuntimeError as e:
    print('Skipping Parquet util demo:', e)

data\processed\util_20250817-142852.parquet
        date ticker       price
0 2024-01-01   AAPL  148.639833
1 2024-01-02   AAPL  148.825222
2 2024-01-03   AAPL  149.635434
3 2024-01-04   AAPL  149.878524
4 2024-01-05   AAPL  149.757249


## 5) Documentation (TODO)
- Update README with a **Data Storage** section (folders, formats, env usage).
- Summarize validation checks and any assumptions.